In [45]:
# clear all the variables
%reset -f
import numpy as np
import pandas as pd
import sys
import graphlab
import os
from os import walk
import scipy.io
import xlrd, os, fnmatch
import pandas as pd
import tensorflow as tf
import plotly.plotly as py
import plotly.graph_objs as go
from scipy.spatial.distance import cdist
# Path variable (where the data is located)
myPath = './data/' 
# Append the path variable to existing search path
sys.path.append(myPath)
# Get the file information in the directory
file_list = []
ignore_list = ['pilot','Expt_I','Expt_II','Expt_III','Expt_IV','Expt_V']
for root, dirs, files in os.walk(myPath):
    print files
    for ig in ignore_list:
        if ig in dirs:
            dirs[:]=[]
    for filename in files:
        if fnmatch.fnmatch(filename.lower(), "*csv"):
            file_list.append(os.path.join(root, filename))

['Adi_06Dec2018_11-43.txt', 'Adi_06Dec2018_11-41.txt', 'pilot_05Dec2018_16-06.csv', '4_07Dec2018_14-34.csv', '.DS_Store', 'Adi_06Dec2018_14-49.txt', 'Adi_06Dec2018_11-09.txt', 'Adi_06Dec2018_10-49.txt', 'Adi_06Dec2018_10-48.txt', 'Adi_06Dec2018_16-03.txt', 'Adi_06Dec2018_10-58.txt', 'pilot_05Dec2018_16-06.txt', 'Adi_06Dec2018_11-01.txt', 'Adi_06Dec2018_10-57.txt', 'Adi_06Dec2018_10-42.txt', 'Adi_06Dec2018_10-56.txt', 'Adi_06Dec2018_11-12.txt', 'Adi_06Dec2018_11-13.txt', '4_07Dec2018_14-34.txt', 'Adi_06Dec2018_11-11.txt', 'Adi_06Dec2018_11-05.txt', 'Adi_06Dec2018_16-03.csv', 'pilot2_06Dec2018_16-00.txt', 'Adi_06Dec2018_14-51.txt', 'Adi_06Dec2018_14-43.txt']


In [49]:
file_list

['./data/pilot_05Dec2018_16-06.csv',
 './data/4_07Dec2018_14-34.csv',
 './data/Adi_06Dec2018_16-03.csv']

In [50]:
data = graphlab.SFrame.read_csv(file_list[1])
temp = data
for i in range(2,len(file_list)):
    temp = graphlab.SFrame.read_csv(file_list[i])
    data = data.append(temp)
len(data)

Finished parsing file /Users/supadhy6/Documents/GitHub/spatio_temporal_information_processing/RSVP_visual_now/data/4_07Dec2018_14-34.csv

Parsing completed. Parsed 100 lines in 0.009726 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,int,str,str,int,int,str,str,int,array,float,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/supadhy6/Documents/GitHub/spatio_temporal_information_processing/RSVP_visual_now/data/4_07Dec2018_14-34.csv

Parsing completed. Parsed 360 lines in 0.009165 secs.

...

Parsing completed. Parsed 100 lines in 0.015577 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,int,str,str,int,int,str,str,int,array,float,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/supadhy6/Documents/GitHub/spatio_temporal_information_processing/RSVP_visual_now/data/Adi_06Dec2018_16-03.csv

Parsing completed. Parsed 360 lines in 0.014264 secs.

RuntimeError: Runtime Exception. Column types are not the same in two SFrames

In [33]:
data['response0'].unique()

dtype: str
Rows: 2
['G', 'B']

In [34]:
data['freq1'] = data.apply(lambda x: 1 if x['response0']=='B' else 0)
data['freq2'] = data.apply(lambda x: 1 if x['response0']=='G' else 0)

In [35]:
import graphlab.aggregate as agg
data['lag'] = data['answerPos1']-data['answerPos0']
data.show()
pcfData = data.groupby(['lag','cueEccentricity'],operations = {'p1':agg.MEAN('freq1'), 'p2':agg.MEAN('freq2')})
pcfData.show()

Canvas is accessible via web browser at the URL: http://localhost:63903/index.html
Opening Canvas in default web browser.
Canvas is accessible via web browser at the URL: http://localhost:63903/index.html
Opening Canvas in default web browser.


In [38]:
pcfData['cueEccentricity'].unique()

dtype: float
Rows: 4
[9.0, 3.0, 4.5, 1.5]

In [39]:
pcfData_1 = pcfData.filter_by([3.0,9.0],'cueEccentricity')
pcfData_2 = pcfData.filter_by([1.5,4.5],'cueEccentricity')

In [43]:
pcfData_1 = pcfData_1.sort(['cueEccentricity','lag'])
trace_data = []
import colorlover as cl
from plotly import tools
colors = cl.scales['3']['seq']['YlOrRd']
for j,ecc in enumerate(pcfData_1['cueEccentricity'].unique().sort()):
        ecc_data = pcfData_1[pcfData_1['cueEccentricity']==ecc]
        trace_data.append(go.Scatter(x = ecc_data['lag'].to_numpy(), 
                                  y = 1-ecc_data['p2'].to_numpy(),name = str(ecc) + 'deg', 
                                     opacity = 1, marker = dict(color = colors[j])))
        
fig = tools.make_subplots(rows=1, cols=1)
fig['layout'].update(barmode='group', title = 'psychometric curve - '+str(len(data)/280)+' participants - ISI (73 msec)')
fig['layout']['xaxis1'].update(title='positional lag of periphery wrt fovea')
fig['layout']['yaxis1'].update(title='Prob. fovea flashed earlier')
for i in range(2):
    fig.append_trace(trace_data[i], 1, 1)
py.iplot(fig, filename='pcf_visual_now')

This is the format of your plot grid:
[ (1,1) x1,y1 ]



In [44]:
pcfData_2 = pcfData_2.sort(['cueEccentricity','lag'])
trace_data = []
import colorlover as cl
from plotly import tools
colors = cl.scales['3']['seq']['YlOrRd']
for j,ecc in enumerate(pcfData_2['cueEccentricity'].unique().sort()):
        ecc_data = pcfData_2[pcfData_2['cueEccentricity']==ecc]
        trace_data.append(go.Scatter(x = ecc_data['lag'].to_numpy(), 
                                  y = 1-ecc_data['p2'].to_numpy(),name = str(ecc) + 'deg', 
                                     opacity = 1, marker = dict(color = colors[j])))
        
fig = tools.make_subplots(rows=1, cols=1)
fig['layout'].update(barmode='group', title = 'psychometric curve - '+str(len(data)/280)+' participants - ISI (73 msec)')
fig['layout']['xaxis1'].update(title='positional lag of periphery wrt fovea')
fig['layout']['yaxis1'].update(title='Prob. fovea flashed earlier')
for i in range(2):
    fig.append_trace(trace_data[i], 1, 1)
py.iplot(fig, filename='pcf_visual_now')

This is the format of your plot grid:
[ (1,1) x1,y1 ]



In [6]:
from sklearn.neighbors import KernelDensity
from scipy.stats import gaussian_kde

In [7]:
X_plot = np.linspace(-10, 10, 10000)
trace_data = []
distributions = []
fill_colors = ['rgba(255,237,160,0.2)', 'rgba(254,178,76,0.2)', 'rgba(240,59,32,0.2)']
for j,ecc in enumerate(cuePosdata['cueEccentricity1'].unique().sort()):
        ecc_data = cuePosdata[cuePosdata['cueEccentricity1']==ecc]
        X = ecc_data['responsePosRelative0'].to_numpy()
        # Gaussian KDE
        #kde = KernelDensity(kernel='gaussian').fit(X)
        kde = gaussian_kde(X)
        log_dens = kde(X_plot)
        distributions.append({'ecc' : ecc, 'dist' : X, 'dens': log_dens})
        trace_data.append(go.Scatter(x = X_plot, y = log_dens,
                                  mode='lines', fill = 'tozeroy', fillcolor = fill_colors[j], name = str(ecc) + 'deg', line = dict(color = colors[j], width = 2) ))
        
fig = tools.make_subplots(rows=1, cols=1)
fig['layout'].update(barmode='group', title = 'performance in RSVP visual now task for different cue positions - '+str(len(data)/280)+' participants - ISI (73 msec)')
fig['layout']['xaxis1'].update(title='position of reported item relative to the true item')
fig['layout']['yaxis1'].update(title='Probability', range = [0,0.8])
for i in range(2):
    fig.append_trace(trace_data[i], 1, 1)
py.iplot(fig, filename='RSVP_visual_now_grouped')


This is the format of your plot grid:
[ (1,1) x1,y1 ]



In [8]:
import numpy as np
from scipy.stats import kurtosis, skew

for x in distributions:
    print( 'Eccentricity: {}'.format( x['ecc']))
    print( 'excess kurtosis of normal distribution (should be 0): {}'.format( kurtosis(x['dens']) ))
    print( 'skewness of normal distribution (should be 0): {}'.format( skew(x['dens']) ))
    print( '\n')

Eccentricity: 2
excess kurtosis of normal distribution (should be 0): 5.32638655219
skewness of normal distribution (should be 0): 2.53956852051


Eccentricity: 6
excess kurtosis of normal distribution (should be 0): 3.05048991029
skewness of normal distribution (should be 0): 2.09537904708


Eccentricity: 10
excess kurtosis of normal distribution (should be 0): 2.69692016
skewness of normal distribution (should be 0): 2.0113438325




In [9]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

hsb = data.to_dataframe()
mod = ols('responsePosRelative0 ~ C(cueEccentricity1)',
                data=hsb).fit()
                
aov_table = sm.stats.anova_lm(mod, typ=2)
print aov_table

                           sum_sq      df         F    PR(>F)
C(cueEccentricity1)    110.326667     2.0  7.398213  0.000618
Residual             46952.351905  6297.0       NaN       NaN


In [10]:
print('Ecc:{}'.format(distributions[0]['ecc'])+' vs.'+ 'Ecc:{}'.format(distributions[1]['ecc']))
scipy.stats.ttest_ind(distributions[0]['dist'], distributions[1]['dist'],axis =0, equal_var=True)

Ecc:2 vs.Ecc:6


Ttest_indResult(statistic=2.524027281944094, pvalue=0.011675146021107764)

In [11]:
print('Ecc:{}'.format(distributions[0]['ecc'])+' vs.'+ 'Ecc:{}'.format(distributions[2]['ecc']))
scipy.stats.ttest_ind(distributions[0]['dist'], distributions[2]['dist'], axis =0,equal_var=True)

Ecc:2 vs.Ecc:10


Ttest_indResult(statistic=2.6030503500762427, pvalue=0.009304938329588968)

In [12]:
print('Ecc:{}'.format(distributions[1]['ecc'])+' vs.'+ 'Ecc:{}'.format(distributions[2]['ecc']))
scipy.stats.ttest_ind(distributions[1]['dist'], distributions[2]['dist'], axis =0,equal_var=True)

Ecc:6 vs.Ecc:10


Ttest_indResult(statistic=0.06890226872235523, pvalue=0.9450739787442742)

In [13]:
import graphlab.aggregate as agg
sub_data = data.groupby(['subject','cueEccentricity1'],operations = {'avg_lag':agg.AVG('responsePosRelative0')})
sub_data = sub_data.sort(['subject','cueEccentricity1'])
skewness = []
kurtosissy = []
for sub in data['subject'].unique().sort():
    subd = data[data['subject']==sub]
    for ecc in subd['cueEccentricity1'].unique().sort():
        eccd = subd[subd['cueEccentricity1']==ecc]
        X = eccd['responsePosRelative0'].to_numpy()
        kde = gaussian_kde(X)
        log_dens = kde(X_plot)
        skewness.append(skew(log_dens))
        kurtosissy.append(kurtosis(log_dens))

sub_data['skew'] = skewness
sub_data['kurtosis'] = kurtosissy
sub_data.show()

Canvas is accessible via web browser at the URL: http://localhost:52995/index.html
Opening Canvas in default web browser.


In [14]:
sub_hsb = sub_data.to_dataframe()
#sub_hsb = hsb.groupby(['subject','cueEccentricity1']).apply(skew('responseRelative0'))
mod = ols('skew ~ C(cueEccentricity1)',
                data=sub_hsb).fit()
                
aov_table = sm.stats.anova_lm(mod)
aov_table

,df,sum_sq,mean_sq,F,PR(>F)
C(cueEccentricity1),2.0,2.960804,1.480402,4.031208,0.025032
Residual,42.0,15.423883,0.367235,NaN,NaN


In [15]:
mod1 = ols('kurtosis ~ C(cueEccentricity1)',
                data=sub_hsb).fit()
                
aov_table1 = sm.stats.anova_lm(mod1)
aov_table1

,df,sum_sq,mean_sq,F,PR(>F)
C(cueEccentricity1),2.0,83.174483,41.587241,3.282586,0.04736
Residual,42.0,532.100097,12.669050,NaN,NaN


In [19]:
deg2_data = sub_data[sub_data['cueEccentricity1']==2]
deg2_data = deg2_data.sort('subject')
deg6_data = sub_data[sub_data['cueEccentricity1']==6]
deg6_data = deg6_data.sort('subject')
deg10_data = sub_data[sub_data['cueEccentricity1']==10]
deg10_data = deg10_data.sort('subject')

deg2 = deg2_data['avg_lag'].to_numpy()
deg6 = deg6_data['avg_lag'].to_numpy()
deg10 = deg10_data['avg_lag'].to_numpy()

In [29]:
print('Ecc:{}'.format(distributions[0]['ecc'])+' vs. zero mean')
scipy.stats.ttest_1samp(deg2,0.0)

Ecc:2 vs. zero mean


Ttest_1sampResult(statistic=0.8138153019881289, pvalue=0.42937571812425135)

In [20]:
print('Ecc:{}'.format(distributions[0]['ecc'])+' vs.'+ 'Ecc:{}'.format(distributions[1]['ecc']))
scipy.stats.ttest_rel(deg2, deg6)

Ecc:2 vs.Ecc:6


Ttest_relResult(statistic=2.8188513016111236, pvalue=0.013663614653794929)

In [21]:
print('Ecc:{}'.format(distributions[0]['ecc'])+' vs.'+ 'Ecc:{}'.format(distributions[2]['ecc']))
scipy.stats.ttest_rel(deg2,deg10)

Ecc:2 vs.Ecc:10


Ttest_relResult(statistic=3.0273766413290124, pvalue=0.009046867282079252)

In [22]:
print('Ecc:{}'.format(distributions[1]['ecc'])+' vs.'+ 'Ecc:{}'.format(distributions[2]['ecc']))
scipy.stats.ttest_rel(deg6,deg10)

Ecc:6 vs.Ecc:10


Ttest_relResult(statistic=-0.029464244105734106, pvalue=0.97691024416633)

In [20]:
output = graphlab.SFrame()
output['subject'] = np.arange(1,16)
output['avg_lag_2deg'] = deg2
output['avg_lag_6deg'] = deg6
output['avg_lag_10deg'] = deg10

RuntimeError: Runtime Exception. Column "avg_lag_2deg" has different size than current columns!

In [ ]:
output

In [ ]:
#output.export_csv('Expt_1_subject_wise_averaged_data.csv')

In [ ]:
data = data.sort(['subject','trialnum'])
data.show()

In [ ]:
raw_deg2_data = data[data['cueEccentricity1']==2]
raw_deg2_data = raw_deg2_data.sort('subject')
raw_deg6_data = data[data['cueEccentricity1']==6]
raw_deg6_data = raw_deg6_data.sort('subject')
raw_deg10_data = data[data['cueEccentricity1']==10]
raw_deg10_data = raw_deg10_data.sort('subject')

raw_deg2 = raw_deg2_data['responsePosRelative0'].to_numpy()
raw_deg6 = raw_deg6_data['responsePosRelative0'].to_numpy()
raw_deg10 = raw_deg10_data['responsePosRelative0'].to_numpy()

In [ ]:
len(raw_deg2)/14

In [ ]:
raw_output = graphlab.SFrame()
raw_output['subject'] = np.repeat(np.arange(1,16),140)
raw_output['lag_2deg'] = raw_deg2
raw_output['lag_6deg'] = raw_deg6
raw_output['lag_10deg'] = raw_deg10

In [ ]:
raw_output.show()

In [ ]:
#raw_output.export_csv('Expt_1_raw_data.csv')

In [ ]:
len(data['subject'])